In [1]:
!pip install git+https://github.com/daavoo/pyntcloud
!pip install ipywidgets
!jupyter nbextension enable --py --sys-prefix widgetsnbextension
!pip install pythreejs
!jupyter nbextension install --py --symlink --sys-prefix pythreejs
!jupyter nbextension enable --py --sys-prefix pythreejs

  Cloning https://github.com/daavoo/pyntcloud to /tmp/pip-req-build-vmllsgt3
  Running setup.py bdist_wheel for pyntcloud ... done
  Stored in directory: /tmp/pip-ephem-wheel-cache-ulgil_67/wheels/be/37/76/afc097c015aa7cdb2ccfc317e1ea8684c387d6b48984dd1ded
Successfully built pyntcloud
Enabling notebook extension jupyter-js-widgets/extension...
      - Validating: OK
  Using cached https://files.pythonhosted.org/packages/b6/0e/f6dbe6e1a9a489cca1bcada3ee0ae5fd49cfb123c6829cc9c1c6583a3fa8/pythreejs-2.0.2-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/2c/9e/16c3cbc63862b36c0b18aa0e1d1dd6a65496d4f8a91a57a728d19bdb74b1/ipydatawidgets-4.0.0-py2.py3-none-any.whl


  Using cached https://files.pythonhosted.org/packages/9c/d1/8d5bd662703cc1764d986f6908a608777305946fa634d34c470cd4a1e729/traittypes-0.2.1-py2.py3-none-any.whl
Installing /usr/local/envs/py3env/lib/python3.5/site-packages/pythreejs/static -> jupyter-threejs
- Validating: OK

    To initialize this nbextension in the browser every time the notebook (or other app) loads:
    
          jupyter nbextension enable pythreejs --py --sys-prefix
    
Enabling notebook extension jupyter-threejs/extension...
      - Validating: OK


In [2]:
import os
import glob
import numpy as np
import pandas as pd
from pyntcloud import PyntCloud

In [5]:
dir_path = "S3DIS/Stanford3dDataset_v1.2/Area_1/conferenceRoom_1"
anno_path = "Annotations/"
file_name = os.path.basename(dir_path) + ".txt"
anno_object_paths = glob.glob(os.path.join(dir_path, anno_path, "*.txt"))

In [4]:
def subsample(points, size=0.05):
  cloud = PyntCloud(points)
  voxelgrid_id = cloud.add_structure("voxelgrid", size_x=size, size_y=size, size_z=size)
  voxelgrid = cloud.structures[voxelgrid_id]
  voxel_centers = pd.DataFrame(data=voxelgrid.voxel_centers)
  voxel_binary = voxelgrid.get_feature_vector(mode="binary").flatten()
  subsampled_points = voxel_centers.assign(binary = pd.DataFrame(voxel_binary))
  subsampled_points = subsampled_points[subsampled_points.binary == 1.0].drop('binary', axis='columns')
  subsampled_points = subsampled_points.reset_index(drop=True)
  subsampled_points = subsampled_points.rename({0:'x', 1:'y', 2:'z'}, axis='columns')
  
  print("{:,} points subsampled to {:,} points for voxel size {}".format(len(cloud.points.index),
                                                                         len(subsampled_points.index),
                                                                         size))
  return subsampled_points

In [5]:
point_cloud = PyntCloud.from_file(os.path.join(dir_path, file_name),
                                 sep=" ",
                                 header=0,
                                 names=["x","y","z","red","green","blue"])

In [15]:
anno_list = []
for file in anno_object_paths:
  df = pd.read_csv(file,
                   sep=" ",
                   header=0,
                   names=["x","y","z","red","green","blue"])
  df = subsample(df, size=0.02)
  if "chair" in os.path.basename(file): #make chairs red  
    df["red"],df["green"],df["blue"] = 200,0,0
  else:
    df["red"],df["green"],df["blue"] = 255,165,0
    

  anno_list.append(df)

anno_list_concat = pd.concat(anno_list, ignore_index=True)
anno_cloud = PyntCloud(anno_list_concat)

4,973 points subsampled to 1,400 points for voxel size 0.02
8,781 points subsampled to 2,445 points for voxel size 0.02
4,662 points subsampled to 1,266 points for voxel size 0.02
6,325 points subsampled to 1,694 points for voxel size 0.02
3,970 points subsampled to 1,060 points for voxel size 0.02
9,873 points subsampled to 2,758 points for voxel size 0.02
4,688 points subsampled to 1,309 points for voxel size 0.02
82,922 points subsampled to 20,877 points for voxel size 0.02
15,239 points subsampled to 4,191 points for voxel size 0.02
5,148 points subsampled to 1,406 points for voxel size 0.02
5,307 points subsampled to 1,470 points for voxel size 0.02
41,344 points subsampled to 11,180 points for voxel size 0.02
19,726 points subsampled to 4,951 points for voxel size 0.02
12,855 points subsampled to 3,566 points for voxel size 0.02
107,351 points subsampled to 26,958 points for voxel size 0.02
190,383 points subsampled to 48,082 points for voxel size 0.02
33,358 points subsampled to

In [10]:
k_neighbors = anno_cloud.get_neighbors(k=10)
anno_cloud.add_scalar_field("normals", k_neighbors=k_neighbors)
anno_cloud.points.columns = ["x","y","z","red","green","blue","nx","ny","nz"]
#orientation_degrees = anno_cloud.add_scalar_field("orientation_degrees")

In [28]:
subsampled_anno_cloud = PyntCloud(subsample(anno_cloud.points, size=0.05))

1,136,585 points subsampled to 49,258 points for voxel size 0.05


In [16]:
#point_cloud.plot(width=1100, height=600, initial_point_size=0.02)
anno_cloud.plot(width=1100, height=600, initial_point_size=0.005)
#anno_cloud.plot(width=1100, height=600, initial_point_size=0.02, use_as_color=orientation_degrees, cmap="jet")
#voxelgrid.plot(d=3, mode="density", cmap="hsv")

Renderer(camera=PerspectiveCamera(aspect=1.8333333333333333, fov=90.0, position=(-17.861522674560547, 80.32363…